# Importing Libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import os
import glob
import matplotlib.pyplot as plt
import math
import cv2


%matplotlib inline

In [2]:
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import statsmodels.api as sm

# Importing Data

In [3]:
complete_data = pd.read_csv("phase1_data.csv")

In [4]:
complete_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   number   95 non-null     int64  
 1   mean_r   95 non-null     float64
 2   mean_g   95 non-null     float64
 3   mean_b   95 non-null     float64
 4   mean_rg  95 non-null     float64
 5   HHR      95 non-null     float64
 6   Ent      95 non-null     float64
 7   B        95 non-null     float64
 8   G1       95 non-null     float64
 9   G2       95 non-null     float64
 10  G3       95 non-null     float64
 11  G4       95 non-null     float64
 12  G5       95 non-null     float64
 13  label    95 non-null     float64
dtypes: float64(13), int64(1)
memory usage: 10.5 KB


In [5]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(complete_data, test_size=0.3)

In [6]:
# Separate features and target variable in train data
X_train = train_data.drop(columns=['number','label'])
y_train = train_data['label']

# Separate features and target variable in test data
X_test = test_data.drop(columns=['number','label'])
y_test = test_data['label']

# Algorithms

In [7]:
algorithms = {
    'Linear Regression': LinearRegression(),
    'SVM Regression': SVR(kernel='poly'),  # Adjust kernel as needed
    'RandomForest': RandomForestRegressor(),
    'Gradient Boost': GradientBoostingRegressor(),
    'knn': KNeighborsRegressor(),
    'LGBM': LGBMRegressor(),
    'CatBoost': CatBoostRegressor(),
    'Kernel Ridge Regressor': KernelRidge(),
    'Elastic Net': ElasticNet(),
    'Bayesian Ridge': BayesianRidge(),
    'XG Boost': XGBRegressor()
}

In [8]:
# Metric tables
metric_table_train = pd.DataFrame()
metric_table_test = pd.DataFrame()

# Training and Testing

In [9]:
# Run the algorithms ... create metrics and plots
for algorithm_name, algorithm in algorithms.items():

    # Train model
    algorithm.fit(X_train, y_train)

    # Train predictions
    y_train_pred = algorithm.predict(X_train)

    # Test predictions
    y_test_pred = algorithm.predict(X_test)

    # Train metrics
    r2_train = r2_score(y_train, y_train_pred)
    mse_train = mean_squared_error(y_train, y_train_pred)
    mae_train = mean_absolute_error(y_train, y_train_pred)

    # Test metrics
    r2_test = r2_score(y_test, y_test_pred)
    mse_test = mean_squared_error(y_test, y_test_pred)
    mae_test = mean_absolute_error(y_test, y_test_pred)

    # Additional metrics using statsmodels for all algorithms
    residuals_train = y_train - y_train_pred
    residuals_test = y_test - y_test_pred

    durbin_watson_stat_train = sm.stats.durbin_watson(residuals_train)
    jb_stat_train, jb_p_value_train, _, _ = sm.stats.jarque_bera(residuals_train)

    durbin_watson_stat_test = sm.stats.durbin_watson(residuals_test)
    jb_stat_test, jb_p_value_test, _, _ = sm.stats.jarque_bera(residuals_test)

    # Update metric tables
    metric_table_train.at[algorithm_name, 'MAE'] = mae_train
    metric_table_train.at[algorithm_name, 'R-squared'] = r2_train
    metric_table_train.at[algorithm_name, 'MSE'] = mse_train
    metric_table_train.at[algorithm_name, 'Durbin-Watson'] = durbin_watson_stat_train
    metric_table_train.at[algorithm_name, 'Jarque-Bera'] = jb_stat_train
    metric_table_train.at[algorithm_name, 'JB P-value'] = jb_p_value_train

    metric_table_test.at[algorithm_name, 'MAE'] = mae_test
    metric_table_test.at[algorithm_name, 'R-squared'] = r2_test
    metric_table_test.at[algorithm_name, 'MSE'] = mse_test
    metric_table_test.at[algorithm_name, 'Durbin-Watson'] = durbin_watson_stat_test
    metric_table_test.at[algorithm_name, 'Jarque-Bera'] = jb_stat_test
    metric_table_test.at[algorithm_name, 'JB P-value'] = jb_p_value_test


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000071 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 275
[LightGBM] [Info] Number of data points in the train set: 66, number of used features: 12
[LightGBM] [Info] Start training from score 11.637879
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

# Results

In [10]:
# Display metrics in tables
print("Metrics - Train Data:\n")
print(metric_table_train.to_string())
print("-------------------------------------------------")

print("Metrics - Test Data:\n")
print(metric_table_test.to_string())

Metrics - Train Data:

                             MAE  R-squared           MSE  Durbin-Watson  Jarque-Bera    JB P-value
Linear Regression       1.307016   0.409645  2.694479e+00       2.126670     0.954674  6.204333e-01
SVM Regression          1.667555  -0.063968  4.856133e+00       1.839129     0.027480  9.863541e-01
RandomForest            0.516591   0.904599  4.354248e-01       2.195539     0.009475  9.952737e-01
Gradient Boost          0.088465   0.997221  1.268571e-02       2.005888     0.447260  7.996110e-01
knn                     1.294545   0.366462  2.891576e+00       1.844759     0.079342  9.611054e-01
LGBM                    0.973033   0.649784  1.598445e+00       2.126255     1.295778  5.231490e-01
CatBoost                0.023427   0.999822  8.108113e-04       2.141189     0.663115  7.178050e-01
Kernel Ridge Regressor  1.320786   0.404205  2.719309e+00       2.155585     1.359377  5.067748e-01
Elastic Net             1.325464   0.401452  2.731874e+00       2.187489     

# To Classification

--> <10.5 --> Anemic

--> >=10.5 ---> Non-Anemic 

In [12]:
algorithm = LinearRegression()

In [13]:
# Train model
algorithm.fit(X_train, y_train)

# Test predictions
y_test_pred = algorithm.predict(X_test)

In [14]:
actual_labels = (y_test<10.5).tolist()
pred_labels = (pd.Series(y_test_pred)<10.5).tolist()

In [15]:
TP=TN=FN=FP = 0
for i in range(len(actual_labels)):
    if(actual_labels[i]==True and pred_labels[i]==True):
        TP +=1
    if(actual_labels[i]==False and pred_labels[i]==False):
        TN +=1
    if(actual_labels[i]==True and pred_labels[i]==False):
        FN +=1
    if(actual_labels[i]==False and pred_labels[i]==True):
        FP +=1

print(f"Accuracy = {(TP+TN)/(TP+TN+FP+FN)}")
print(f"Precision = {(TP)/(TP+FP)}")
print(f"Sensitivity = {(TP)/(TP+FN)}")
print(f"Specificity = {(TN)/(TN+FP)}")

Accuracy = 0.7241379310344828
Precision = 0.7
Sensitivity = 0.5833333333333334
Specificity = 0.8235294117647058


# LEAVE ONE OUT

In [11]:
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import absolute
from numpy import sqrt

In [12]:
cv = LeaveOneOut()

In [13]:
X = complete_data.drop(columns=['number','label'])
y = complete_data['label']

## XGB Regression

In [14]:
model = XGBRegressor()

#use LOOCV to evaluate model
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error',
                         cv=cv, n_jobs=-1)

In [15]:
y_true = y
y_pred = y + scores

In [16]:
print(f"MAE {mean(absolute(scores))}")
print(f"RMSE {sqrt(mean(absolute(scores)))}")
print(f"R-squared {r2_score(y_true,y_pred)}")

MAE 1.4623021316528322
RMSE 1.2092568509844517
R-squared 0.17594592086729732


## CatBoost Regression

In [30]:
model = CatBoostRegressor()

#use LOOCV to evaluate model
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error',
                         cv=cv, n_jobs=-1)

In [31]:
y_true = y
y_pred = y + scores

In [32]:
print(f"MAE {mean(absolute(scores))}")
print(f"RMSE {sqrt(mean(absolute(scores)))}")
print(f"R-squared {r2_score(y_true,y_pred)}")

MAE 1.3620070698060949
RMSE 1.1670505857957036
R-squared 0.2765393867638387


## RandomForest

In [17]:
model = RandomForestRegressor()

#use LOOCV to evaluate model
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error',
                         cv=cv, n_jobs=-1)

In [18]:
y_true = y
y_pred = y + scores

In [19]:
print(f"MAE {mean(absolute(scores))}")
print(f"RMSE {sqrt(mean(absolute(scores)))}")
print(f"R-squared {r2_score(y_true,y_pred)}")

MAE 1.3156000000000003
RMSE 1.1469960767151737
R-squared 0.33053261704678105


## LGBM

In [20]:
model = LGBMRegressor()

#use LOOCV to evaluate model
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error',
                         cv=cv, n_jobs=-1)

In [21]:
y_true = y
y_pred = y + scores

In [22]:
print(f"MAE {mean(absolute(scores))}")
print(f"RMSE {sqrt(mean(absolute(scores)))}")
print(f"R-squared {r2_score(y_true,y_pred)}")

MAE 1.4042862084277188
RMSE 1.1850258260593811
R-squared 0.25891560759393517


## Linear Regression

In [23]:
model = LinearRegression()

#use LOOCV to evaluate model
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error',
                         cv=cv, n_jobs=-1)

In [24]:
y_true = y
y_pred = y + scores

In [25]:
print(f"MAE {mean(absolute(scores))}")
print(f"RMSE {sqrt(mean(absolute(scores)))}")
print(f"R-squared {r2_score(y_true,y_pred)}")

MAE 1.4203990757744822
RMSE 1.1918049654932985
R-squared 0.16455675116161728


# PyCaret

In [27]:
import pycaret

In [28]:
from pycaret.regression import *
s = setup(complete_data, target='label', ignore_features=['number'], preprocess=False, session_id=123)

,Description,Value
0,Session id,123
1,Target,label
2,Target type,Regression
3,Original data shape,"(95, 14)"
4,Transformed data shape,"(95, 13)"
5,Transformed train set shape,"(66, 13)"
6,Transformed test set shape,"(29, 13)"
7,Ignore features,1
8,Numeric features,12


In [29]:
best_r = compare_models(sort = 'R2', n_select = 5)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,1.2932,2.8632,1.6091,0.2015,0.1331,0.1202,0.0690
br,Bayesian Ridge,1.2679,2.5359,1.5397,0.1846,0.1259,0.1152,0.0130
lasso,Lasso Regression,1.2836,2.5546,1.5491,0.1827,0.1265,0.1162,0.0160
llar,Lasso Least Angle Regression,1.2840,2.5553,1.5493,0.1825,0.1265,0.1163,0.0130
en,Elastic Net,1.2645,2.5409,1.5421,0.1750,0.1259,0.1145,0.0100
ridge,Ridge Regression,1.3407,2.7258,1.5867,0.1292,0.1311,0.1222,0.0120
lightgbm,Light Gradient Boosting Machine,1.3939,3.1547,1.6551,0.1121,0.1337,0.1268,0.0410
rf,Random Forest Regressor,1.3935,3.0361,1.6632,0.1113,0.1359,0.1276,0.1020
huber,Huber Regressor,1.3159,2.7659,1.6031,0.0915,0.1312,0.1189,0.0240
lr,Linear Regression,1.3962,3.1181,1.6866,0.0572,0.1547,0.1292,1.3550
